## 10k immu PBMC

To increase the amount of data i wil also add the PBMC sets. Most is copy paste from the whole_blood set

Open TODOs:
- check if microarray and PBMC is mergeable:
    - Flo emails
    - online research 
    - write down preproc pipeline from 10k 
    - compare manhattans
- add study_accession information, (add platform ... ) 
- clean up whole folder structure


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
from utils.helper import violinplot_overall, scatter_plot, manhattanplot, get_negative_values, METADATA_COLS


In [ ]:
ten_k_prefix="/var/datasets/10kImmunomes/transcriptomics/"
print("\n###########################################################")
print("rna_seq_pbmc")
print("============================================================")
rna_seq_pbmc = pd.read_csv(f"{ten_k_prefix}/pbmc_rna_counts.csv",sep=",")
print(rna_seq_pbmc.columns)
print(rna_seq_pbmc.shape)


print("\n###########################################################")
print("marray_pbmc")
print("============================================================")
marray_pbmc = pd.read_csv(f"{ten_k_prefix}/gene_pbmc_formatted.csv",sep=",")
print(marray_pbmc.columns)
print(marray_pbmc.shape)


### Microarray PBMC

Exact same method as Whole blood: copy paste

In [ ]:
####Microarray Data: Gene expression array data were obtained in three formats.
# Data in their original formats (.CEL files, series matrix files, etc) constitute the “raw” gene expression data.

# For data collected on Affymetrix platforms, we utilized the ReadAffy utility in the affy Bioconductor package to read in raw .CEL files. 
# The rma utility was used to conduct Robust Multichip Average (rma) background correction (as in (4)), quantile normalization, and log2 normalization of the data. 
##### CRITERIA: PLATFORM_DESCRIPTION contains 'Affymetrix'

# For data collected on Illumina platforms and stored in the Gene Expression Omnibus (GEO) database, we utilized the getGEO utility in the 
# GEOquery Bioconductor package to read the expression files and the preprocessCore package to conduction rma background correction, quantile normalization, and log2 normalization of the gene expression data.
##### CRITERIA: PLATFORM_DESCRIPTION contains 'illumina'

# Finally, for data collected on Illumina platforms but not stored in GEO, we utilized the read.ilmn utility of the limma Bioconductor package 
# to read in the data, and the neqc function to rma background correct, quantile normalize, and log2 normalize the gene expression data.
##### CRITERIA: Study has GSMxxxxxxxx information for study ? 

# In all instances, probe IDs were converted to Entrez Gene IDs. Where multiple probes mapped to the same Entrez Gene ID, 
# the median value across probes was used to represent the expression value of the corresponding gene.
# The background-corrected and normalized datasets were combined based on common Entrez IDs, missing values were imputed with a 
# k-nearest neighbors algorithm (R package: impute, function: impute.knn) using k = 10 and default values for rowmax, colmax, and maxp. 
# Enter Gene IDs were then converted to HUGO gene names. 
# These steps together produced the “formatted” gene expression files.

# To create the “formatted and normalized” datasets, we utilized established empirical Bayes algorithm ComBat for batch correction (2), compensating for possible batch effects while maintaining potential effects of age, race, and sex across datasets.

### okay das is schonmal nicht so einheitlich wie erhofft. 
### am besten sind die microarrays
print(f'Studies: {marray_pbmc.shape}')
print(f'Studies: {len(marray_pbmc['study_accession'].unique())}')
print(f'Subjects: {len(marray_pbmc['subject_accession'].unique())}')
print(f'Data ACCESSIONS ??: {len(marray_pbmc['data_accession'].unique())}')
marray_pbmc.head()

In [ ]:
print(marray_pbmc['study_accession'].unique())
#SDY269 got 251 different REPOSITORY_ACCESSIONS -> individual subject ... its like GSM734016 -> when u enter into GEO you can get the platform (GPL13158)
mapping_platform_geo_id = {
    'SDY269':'GPL13158',
    'SDY33':'?', # keinen immport eintrag
    'SDY404':'GPL10558',
    'SDY420':'GPL4133',
    'SDY63':'GPL10558',
    'SDY720':'GPL10558',
    'SDY816':'?',  # keinen immport eintrag
                    }	
mapping_platform_description = {
    'SDY269':'[HT_HG-U133_Plus_PM] Affymetrix HT HG-U133+ PM Array Plate',
    'SDY33':'Affymetrix CEL',
    'SDY404':'Illumina HumanHT-12 V4.0 expression beadchip',
    'SDY420':'Agilent-014850 Whole Human Genome Microarray 4x44K G4112F (Feature Number version)',
    'SDY63':'Illumina HumanHT-12 V4.0 expression beadchip',
    'SDY720':'Illumina HumanHT-12 V4.0 expression beadchip',
    'SDY816':'Illumina Beadarray',
                    }	

marray_pbmc['PLATFORM_GEO_ID'] = marray_pbmc['study_accession'].apply(lambda x: mapping_platform_geo_id.get(x))
marray_pbmc['PLATFORM_DESCRIPTION'] = marray_pbmc['study_accession'].apply(lambda x: mapping_platform_description.get(x))
marray_pbmc.head()



In [ ]:
#REORDER COLUMNS
cols = marray_pbmc.columns.tolist()
cols = cols[-2:] + cols[:-6]
marray_pbmc = marray_pbmc[cols]
marray_pbmc = marray_pbmc.drop_duplicates()
marray_pbmc['gender'] = marray_pbmc['gender'].apply(lambda x: x.lower() if isinstance(x, str) else x)




In [ ]:
marray_pbmc_metadata = marray_pbmc.iloc[:,:8]
marray_pbmc_wo_meta =  marray_pbmc.iloc[:,8:]
marray_pbmc_wo_meta.columns

In [ ]:
marray_pbmc.describe()
print(get_negative_values(marray_pbmc))
### min values smaller 0 ... 
# my approach: reverse log2 transform (2^data) then add 1 then log2 transform 

marray_pbmc_wo_meta_untransformed = np.exp2(marray_pbmc_wo_meta)
marray_pbmc_wo_meta_untransformed = marray_pbmc_wo_meta_untransformed+1
marray_pbmc_wo_meta_non_neg = np.log2(marray_pbmc_wo_meta_untransformed)

# Ensure no more neg values
print(get_negative_values(marray_pbmc_wo_meta_non_neg))



In [ ]:
marray_pbmc = marray_pbmc_metadata.join(marray_pbmc_wo_meta_non_neg)
marray_pbmc.to_csv(f'pbmc/marray_pbmc_prepared.csv')
marray_pbmc.head()


In [ ]:
manhattanplot(marray_pbmc, start_col=8)

In [ ]:
violinplot_overall(marray_pbmc,  x_topic='gender', y_topic='age', hue_split='gender', bin_name='pbmc')

In [ ]:
marray_pbmc.describe()

## This actually looks good... 

# RNAseq PBMC

already got stuff from 10k_immu notebook... 

In [ ]:
#### RNA-Seq Whole Blood and PBMC Data: 
# The RNA-Seq peripheral mononucler cells comes from Immport study SDY67.
# The files used from Immport were formatted and normalized by the EPIC Consortium. 
#### This apparently made the FKPM transofrmation. -> TPM
# The whole blood datasets were converted into TPM files and harmonized together using the HarmonyRNA algorithm. 
#### 
# More information on the study can be found in this Nature Article published by Amy H. Lee, et al. 
# Some minor additional formatting was done for presenting data on 10k Immunomes.
print('RNAseq PBMC: ')
rna_seq_pbmc_T = rna_seq_pbmc.T
rna_seq_pbmc_T.reset_index(0, inplace=True)
rna_seq_pbmc_T.columns = rna_seq_pbmc_T.iloc[0]
rna_seq_pbmc_T = rna_seq_pbmc_T.drop(rna_seq_pbmc_T.index[0])
### metadata is separate here
rnaseq_pbmc_metadata = pd.read_csv(ten_k_prefix+'rnaseq_pbmc_metadata.csv')
rna_seq_pbmc_T = rna_seq_pbmc_T.rename(columns={'hgnc': 'Subject'})
rna_seq_pbmc_T_w_meta = pd.merge(rna_seq_pbmc_T, rnaseq_pbmc_metadata, on='Subject', how='left')

## Rearrange columns: 
cols = rna_seq_pbmc_T_w_meta.columns.tolist()
cols = cols[-4:] + cols[:-4]
rna_seq_pbmc_T_w_meta = rna_seq_pbmc_T_w_meta[cols]
## nice ! 
print(f'Genes: {rna_seq_pbmc_T_w_meta.shape}')
print(f'Studies: {len(rna_seq_pbmc_T_w_meta['Study'].unique())}')
print(f'Subjects: {len(rna_seq_pbmc_T_w_meta['Subject'].unique())}')
### only one study (SDY67)
rna_seq_pbmc_T_w_meta['PLATFORM_GEO_ID'] = 'GPL11154'
rna_seq_pbmc_T_w_meta['PLATFORM_DESCRIPTION'] = 'Illumina HiSeq 2500 (Homo sapiens)'
## Rearrange columns: 
cols = rna_seq_pbmc_T_w_meta.columns.tolist()
cols = cols[-2:] + cols[:-2]
rna_seq_pbmc_T_w_meta = rna_seq_pbmc_T_w_meta[cols]
rna_seq_pbmc_T_w_meta = rna_seq_pbmc_T_w_meta.drop_duplicates()
rna_seq_pbmc_T_w_meta.head()

### SDY67 
### hat keine repo ID ... 

#### (own) Preprocessing
match manhattan plots, similar as in whole blood

In [ ]:
## Step1: Quant. normalisation
import qnorm
rna_seq_pbmc_T_wo_meta = rna_seq_pbmc_T_w_meta.iloc[:, 7:]
metadata = rna_seq_pbmc_T_w_meta.iloc[:, :7]
rna_seq_pbmc_T_wo_meta_quant = qnorm.quantile_normalize(rna_seq_pbmc_T_wo_meta, axis=0).astype(float)
print(rna_seq_pbmc_T_wo_meta_quant.columns)

## Step2: log2 normalization
rna_seq_pbmc_T_wo_meta_quant = rna_seq_pbmc_T_wo_meta_quant+1.
rna_seq_pbmc_T_wo_meta_log2 = np.log2(rna_seq_pbmc_T_wo_meta_quant) 
rna_seq_pbmc_T_wo_meta_log2 = rna_seq_pbmc_T_wo_meta_log2.astype(float)
rna_seq_pbmc_T_wo_meta_log2.describe()

In [ ]:
manhattanplot(rna_seq_pbmc_T_wo_meta_log2, start_col=0)

In [ ]:
### get metadata back into CSV
rna_seq_pbmc_T_w_meta = rna_seq_pbmc_T_w_meta.rename(columns={'Age': 'age', 'Sex': 'gender', 
                                                              'Study':'study_accession', 
                                                              'Expression': 'data_accession', 
                                                            'Subject': 'subject_accession'})
rna_seq_pbmc_T_w_meta_log2 = metadata.join(rna_seq_pbmc_T_wo_meta_log2)
rna_seq_pbmc_T_w_meta_log2.to_csv(f'pbmc/rna_seq_pbmc_T_w_meta_preproc.csv')


In [ ]:
rna_seq_pbmc_T_w_meta_log2.head()

In [ ]:
description = rna_seq_pbmc_T_w_meta_log2.describe().T
negative_mins = description[description['min'] < 0]
print(negative_mins)
#### no negative values here 


In [ ]:
violinplot_overall(rna_seq_pbmc_T_w_meta_log2,  x_topic='Sex', y_topic='Age', hue_split='Sex', bin_name='pbmc')

## MERGE area

In [ ]:
rna_seq_pbmc_T_w_meta_log2.shape
rna_seq_pbmc_T_w_meta_log2.describe()

In [ ]:
marray_pbmc.shape
marray_pbmc.describe()

In [ ]:
## Step1: rename stuff so they match each other (and also whole blood .csv table....)
rna_seq_pbmc_T_w_meta_log2['METHOD']='rnaseq'
marray_pbmc['METHOD']='array'

common_columns = rna_seq_pbmc_T_w_meta_log2.columns.intersection(marray_pbmc.columns)
# put metadtaa together
common_columns = common_columns.to_list()
last_element = common_columns.pop()
common_columns.insert(0, last_element)


combi_df_pbmc = pd.concat([marray_pbmc, rna_seq_pbmc_T_w_meta_log2], ignore_index=True)
combi_df_pbmc['gender'] = combi_df_pbmc['gender'].str.lower()
combi_df_pbmc['TYPE']='pbmc'
print(combi_df_pbmc.shape)


### Remove possible duplicated subjects and entries with Nan 
combi_df_pbmc_reduced = combi_df_pbmc.drop_duplicates()
combi_df_pbmc_reduced = combi_df_pbmc_reduced.drop_duplicates(subset=['subject_accession'], keep='first')
print(combi_df_pbmc_reduced.shape)


## drop entries that dont fullfill those min metadata
essential_columns = ['age', 'gender', 'study_accession']
combi_df_pbmc_reduced = combi_df_pbmc_reduced.dropna(subset=essential_columns, axis=0, how='any')
print(combi_df_pbmc_reduced.shape)

# Handle NA values
#combi_df_whole_blood_reduced = combi_df_whole_blood_reduced.dropna(axis=1, how='any') 
#print(combi_df_whole_blood_reduced.shape)

combi_df_pbmc.head()


In [ ]:
combi_df_pbmc.to_csv(f'pbmc/combi_df_pbmc_all.csv')


In [ ]:
no_meta_cols = list(set(common_columns) - set(METADATA_COLS))
print(len(no_meta_cols))
new_order_cols = METADATA_COLS + no_meta_cols
print(len(new_order_cols))

In [ ]:
combi_df_pbmc_common_cols = combi_df_pbmc[new_order_cols]
combi_df_pbmc_common_cols.to_csv(f'pbmc/combi_df_pbmc_ordered_common_cols.csv')


In [ ]:
violinplot_overall(combi_df_pbmc_common_cols,  x_topic='gender', y_topic='age', hue_split='gender', bin_name='pbmc')

In [ ]:
print(combi_df_pbmc_common_cols.shape)
combi_df_pbmc_common_cols.head()

In [ ]:
combi_df_pbmc_common_cols.describe()

In [ ]:
manhattanplot(combi_df_pbmc_common_cols, start_col=9)

In [ ]:
combi_df_pbmc_common_cols[no_meta_cols] = combi_df_pbmc_common_cols[no_meta_cols].astype(float)

In [ ]:
## Does not yet work as only age is float at the moment
description = combi_df_pbmc_common_cols[combi_df_pbmc_common_cols['METHOD']=='rnaseq'].describe().T
negative_mins = description[description['min'] < 0]
print(negative_mins)
combi_df_pbmc_common_cols.describe()

# Paper  Enrichment score
### Problem: Needs gene sets with related genes ... 
### Increase comparability between microarray and RNAseq
https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1008295 <- hier wird eben gezeigt wie der score nützlich ist (Spearson corr index almost 1)>
https://www.nature.com/articles/nature08460 <- des is der eigentliche enrichment score paper >
transformed gene expression values to gene set enrichment scores (ES) by mapping the genes on preselected groups of genes. Gene sets are groups of genes that have been experimentally and/or computationally related to a biological function
involves the conversion from gene expression levels to ranks